In [1]:
import os
from dotenv import load_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from IPython.display import display,Markdown

In [3]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Define a question about apple quantities and usage.
question = """The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""

# Provide context for answering questions, emphasizing detailed math and reasoning.
context = """Answer questions showing the full math and reasoning. Follow the pattern in the example."""

# Example problem-solving scenario involving arithmetic calculations.
one_shot_example = """Example 
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he 
have now? 

A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: """

In [5]:
# Create a planning pipeline using PromptTemplate, ChatOpenAI, StrOutputParser, and RunnablePassthrough.
planner = (
    
    # Combine context and example with an input placeholder for dynamic prompt creation.
    PromptTemplate.from_template(context + one_shot_example + " {input}")
    
    # Process the combined template through ChatOpenAI for text generation.
    | ChatOpenAI()
    
    # Parse the generated text into structured data.
    | StrOutputParser()
    
    # Wrap the parsed output in a RunnablePassthrough object for further processing.
    | {"base_response": RunnablePassthrough()}
)

In [6]:
# Generate a structured response using PromptTemplate, ChatOpenAI, and StrOutputParser.
answer_1 = (
    
    # Start with a base response template followed by "A: ".
    PromptTemplate.from_template("{base_response} A: ")
    
    # Use ChatOpenAI with temperature set to 0 for deterministic output.
    | ChatOpenAI(temperature=0)
    
    # Parse the generated text into structured data.
    | StrOutputParser()
)

In [7]:
# Generate a structured response using PromptTemplate, ChatOpenAI, and StrOutputParser.
answer_2 = (
    
    # Start with a base response template followed by "A: ".
    PromptTemplate.from_template("{base_response} A: ")
    
    # Use ChatOpenAI with temperature set to 0 for deterministic output.
    | ChatOpenAI(temperature=0.1)
    
    # Parse the generated text into structured data.
    | StrOutputParser()
)

In [8]:
# Generate a structured response using PromptTemplate, ChatOpenAI, and StrOutputParser.
answer_3 = (
    
    # Start with a base response template followed by "A: ".
    PromptTemplate.from_template("{base_response} A: ")
    
    # Use ChatOpenAI with temperature set to 0 for deterministic output.
    | ChatOpenAI(temperature=0.7)
    
    # Parse the generated text into structured data.
    | StrOutputParser()
)

In [9]:
# Final responder setup for formatting and displaying results in markdown.
final_responder = (
    
    # Template for outputting results in markdown format.
    PromptTemplate.from_template(
        """Output all the final results in this markdown format: Result 1: {results_1} \n Result 2:{results_2} \n Result 3: 
        {results_3}"""
    )
    
    # Process the template through ChatOpenAI for text generation.
    | ChatOpenAI()
    
    # Parse the generated text into structured data.
    | StrOutputParser()
)

In [10]:
# Chain together components for a comprehensive response process.
chain = (
    
    # Initial planning stage.
    planner
    
    # Map results and original response to specific variables.
    | {
        "results_1": answer_1,
        "results_2": answer_2,
        "results_3": answer_3,
        "original_response": itemgetter("base_response"),
    }
    
    # Finalize the response with formatting and display.
    | final_responder
)

In [11]:
# Invoke the response chain with a given input question.
answers = chain.invoke({"input": question})

# Display the generated answers in Markdown format.
display(Markdown(answers))

Result 1: 9 apples. 

Result 2: The cafeteria had 9 apples left after lunch. 

Result 3: 
        The cafeteria had 9 apples left after lunch.